In [ ]:
#WEB SCRAPE USING REGULAR EXPRESSION

from urllib.request import urlopen
import re

In [ ]:
url = "https://vnr500.com.vn/Charts/Index?chartId=1"
page = urlopen(url)
html_byte = page.read()
html = html_byte.decode("utf-8")

In [ ]:
#INDUSTRY LIST

industry = []
industry_pattern = "nghe/.*?-Post"
industry_entry = re.findall(industry_pattern, html, re.IGNORECASE)
for entry in industry_entry:
    entry = entry.replace("-"," ")
    entry = entry.replace("nghe/","")
    entry = entry.replace("Post","")
    industry.append(entry)

In [ ]:
#CEO LIST

CEO_list_1 = []
CEO_count = 0 
CEO_pattern = "profile.*</a"
CEO_entry = re.findall(CEO_pattern, html, re.IGNORECASE)
CEO_list = []
for entry in CEO_entry:
    new_pattern = ">.*?<"
    new_entry = re.findall(new_pattern,entry,re.IGNORECASE)
    CEO_list_1.append(new_entry)
for entry1 in CEO_list_1:
    entry1[0] = entry1[0].replace(">","")
    entry1[0] = entry1[0].replace("<","")
    CEO_list.append(entry1[0])

In [ ]:
#COMPANY NAME LIST

company_list = []
company_count = 0
company_name_pattern = "nghiep.*-Chart"
company_entry = re.findall(company_name_pattern,html, re.IGNORECASE)
for entry in company_entry:
    entry = re.sub("nghiep/", "", entry)
    entry = re.sub("-", " ", entry)
    entry = re.sub("Chart", "", entry)
    if entry not in company_list and company_count <= 499:
        company_list.append(entry)
        company_count += 1


In [ ]:
#RANKING, TAX CODE, TICKER, HQ ADDRESS, PHONE, TAX, EMAIL, WEBSITE, CAPITAL, & ESTABLISHMENT YEAR FROM INDIVIDUAL COMPANY PROFILES

company_list2 = []
company_count2 = 0
company_name_pattern2 = "nghiep/.*.html"
company_entry2 = re.findall(company_name_pattern2,html, re.IGNORECASE)

for entry in company_entry2:
    if entry not in company_list2 and company_count2 <= 499:
        company_list2.append(entry)
        company_count2 += 1

ranking = []
tax_code = []
ticker = []
hq_address = []
phone = []
fax = []
email = []
website = []
capital = []
establish = []

for entry in company_list2:
    link_pattern = "https://vnr500.com.vn/Thong-tin-doanh-" + entry
    try:
        profile = urlopen(link_pattern)
        p_html = profile.read()
        profile_html = p_html.decode("utf-8")
    except UnicodeEncodeError:
        pass
    clean_list = []
    info_pattern = "<td>(?s).*?</td>"
    info_entry = re.findall(info_pattern, profile_html, re.IGNORECASE)
    for entry in info_entry:
        clean_list.append(entry)
    ranking.append(clean_list[0])
    tax_code.append(clean_list[1])
    ticker.append(clean_list[2])
    hq_address.append(clean_list[3])
    phone.append(clean_list[4])
    fax.append(clean_list[5])
    email.append(clean_list[6])
    web_pattern = "<td><a href=.*?>"
    web_entry = re.findall(web_pattern, profile_html)
    website.append(web_entry[0])
    cap_count = 0
    cap_pattern = '''class="ct_box_content_2">(?s).*?</div>'''
    cap_entry = re.findall(cap_pattern, profile_html, re.IGNORECASE)
    for entry in cap_entry:
       if cap_count == 0:
            capital.append(entry)
            cap_count += 1
    establish.append(clean_list[10])
    

In [ ]:
#CLEAN THE ACQUIRED LIST

def clean_entry(list):
    cleaned_list = []
    for entry in list:
        entry = entry.replace('<td>','')
        entry = entry.replace('\n','')
        entry = entry.replace('</td>','')
        entry = entry.replace('''class="ct_box_content_2">''',"")
        entry = entry.replace("</div>","")
        entry = entry.replace("VNĐ","")
        entry = entry.replace('''class="ct_box_content_2">''',"")
        entry = entry.replace("</div>","")
        entry = entry.replace(">","")
        entry = entry.replace("<a href=","")
        entry = entry.replace("Chưa ni&#234;m yết",'Unlisted')
        entry = entry.strip()
        cleaned_list.append(entry)
    for i, entry in enumerate(cleaned_list):
        list[i] = entry
    
   

In [ ]:
clean_entry(ranking)
clean_entry(tax_code)
clean_entry(ticker)
clean_entry(hq_address)
clean_entry(phone)
clean_entry(fax)
clean_entry(email)
clean_entry(website)
clean_entry(capital)
clean_entry(establish)

In [ ]:
#EXPORT TO FILE

import pandas as pd

dict = {"Company Name":company_list, "Website" : website, "Industry" : industry, "CEO":CEO_list, "Ranking": ranking, "Tax Code": tax_code, "Ticker": ticker, "HQ Address": hq_address, "Phone": phone, "Fax": fax, "Email" : email, "Capital" : capital, "Established" : establish}

df = pd.DataFrame(dict)
df.to_csv('top500.csv')
